# Import Library
Library python adalah kumpulan modul terkait berisi kumpulan kode yang dapat digunakan berulang kali dalam program yang berbeda. Adanya library membuat pemrograman python menjadi lebih sederhana dan nyaman bagi programmer karena tidak perlu menulis kode yang sama berulang kali untuk program yang berbeda. maka dari itu terlebih dahulu kita perlu untuk melakukan import yang diperlukan, berikut ini import yang diperlukan

In [19]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
import nltk 
import string
import re


# Crawling Data
Data Crawling adalah prosedur pengumpulan data besar yang dapat menjelajah hingga ke halaman web paling dalam. Library yang saya gunakan untuk melakukan crawling data adalah library scrapy. Untuk melakukan crawling data, yang diperlukan antara lain selector html dari data yang akan di crawling. code untuk crawling ini harus dijalankan pada terminal dengan cara mengetikkan "scrapy namafile.py -o namafile.csv". Berikut ini code untuk melakukan crawling data pada web pta.trunojoyo.ac.id

In [20]:
import scrapy

class crawling(scrapy.Spider):
    name = "pta"

    def start_requests(self):
        x = 100000
        for i in range (1,200):
            x += 1
            urls = [
                'https://pta.trunojoyo.ac.id/welcome/detail/040411' + str(x),
                'https://pta.trunojoyo.ac.id/welcome/detail/070411' + str(x),
                'https://pta.trunojoyo.ac.id/welcome/detail/080411' + str(x),
            ]
            for url in urls:
                yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # print(response.url)
        yield{
            'judul': response.css('#content_journal > ul > li > div:nth-child(2) > a::text').extract(),
            'abstraksi': response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(2) > p::text').extract(),
        }

# Membaca Data Crawling
Agar data data yang sudah kita crawling tadi bisa terbaca dalam code, maka kita perlu untuk memanggil library pandas lalu memasukkan nama file hasil data crawling kita tadi. Berikut code yang diperlukan

In [21]:
data = pd.read_csv('crawling.csv')
data.head()

,judul,abstraksi
0,Pengembangan Sumber Daya Manusia Dalam Rangka ...,ABSTRAK\r\nPenelitian ini bertujuan untuk mene...
1,NaN,NaN
2,PENGARUH ATRIBUT PRODUK TERHADAP KEPUTUSAN PEM...,Abstrak\r\n\r\nTujuan penelitian ini adalah un...
3,NaN,NaN
4,NaN,NaN


# Menghapus Baris yang Kosong
Pada data yang telah kita crawling, setelah di cek ternyata banyak terdapat baris yang kosong. Untuk itu perlu kita hapus baris-baris yang kosong tersebut. berikut ini code untuk menghapus baris yang kosong

In [22]:
data.dropna(inplace=True)
data.isnull().sum()
data.head()

,judul,abstraksi
0,Pengembangan Sumber Daya Manusia Dalam Rangka ...,ABSTRAK\r\nPenelitian ini bertujuan untuk mene...
2,PENGARUH ATRIBUT PRODUK TERHADAP KEPUTUSAN PEM...,Abstrak\r\n\r\nTujuan penelitian ini adalah un...
6,PENGARUH IKLAN ONLINE MELALUI MEDIA FACEBOOK T...,"Abstrak\r\n\r\nHening Ary Putra, Pengaruh Ikla..."
8,PENGARUH UNSUR - UNSUR KOMUNIKASI TERHADAP PRO...,ABSTRAK\r\n\r\n\tPenelitian ini bertujuan untu...
11,PENGARUH DEBT TO TOTAL ASSET RATIO DAN CURRENT...,Abstrak\r\n\r\nPendekatan penelitian yang digu...


# Menghapus Baris yang Tidak Diperlukan
Dalam permasalahan kali ini, data yang diperlukan hanya data abstraksi saja sedangkan data judul tidak diperlukan. Untuk itu perlu kita hapus kolom judul. Berikut ini code untuk menghapus kolom judul

In [23]:
data.drop(['judul'],axis=1,inplace=True)
data.head()

,abstraksi
0,ABSTRAK\r\nPenelitian ini bertujuan untuk mene...
2,Abstrak\r\n\r\nTujuan penelitian ini adalah un...
6,"Abstrak\r\n\r\nHening Ary Putra, Pengaruh Ikla..."
8,ABSTRAK\r\n\r\n\tPenelitian ini bertujuan untu...
11,Abstrak\r\n\r\nPendekatan penelitian yang digu...


# Case Folding, Tokenizing, Stopword, dan Stemming
Case folding adalah metode untuk mengubah semua huruf dalam dokumen menjadi huruf kecil. Tokenizing adalah proses pemisahan teks menjadi potongan-potongan yang disebut sebagai token untuk kemudian di analisa. Filtering adalah tahap mengambil kata-kata penting dari hasil token dengan menggunakan algoritma stoplist (membuang kata kurang penting) atau wordlist (menyimpan kata penting). Stemming adalah proses menghilangkan infleksi kata ke bentuk dasarnya, namun bentuk dasar tersebut tidak berarti sama dengan akar kata (root word).

In [24]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet abstraksi "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
data['abstraksi_clean'] = data['abstraksi'].apply(lambda x: clean_tweets(x))
data.head()

,abstraksi,abstraksi_clean
0,ABSTRAK\r\nPenelitian ini bertujuan untuk mene...,"[abstrak, teliti, tuju, teliti, didik, latih, ..."
2,Abstrak\r\n\r\nTujuan penelitian ini adalah un...,"[abstrak, tuju, teliti, identifikasi, variabel..."
6,"Abstrak\r\n\r\nHening Ary Putra, Pengaruh Ikla...","[abstrak, hening, ary, putra, pengaruh, iklan,..."
8,ABSTRAK\r\n\r\n\tPenelitian ini bertujuan untu...,"[abstrak, teliti, tuju, pengaruh, unsur, unsur..."
11,Abstrak\r\n\r\nPendekatan penelitian yang digu...,"[abstrak, dekat, teliti, teliti, dekat, kuanti..."


# Remove Punctuation 
Setelah kita melakukan proses diatas sebelumnya, hasil yang diperoleh memberikan banyak tambahan tanda baca, untuk itu perlu kita hapus tanda baca tersebut. Remove Punctuation adalah proses dimana sistem akan menghilangkan tanda baca atau simbol yang ada dalam dataset. Berikut code yang dibutuhkan

In [25]:
def remove_punct(abstraksi):
    abstraksi  = " ".join([char for char in abstraksi if char not in string.punctuation])
    return abstraksi

data['abstraksi_akhir'] = data['abstraksi_clean'].apply(lambda x: remove_punct(x))
data.head()

,abstraksi,abstraksi_clean,abstraksi_akhir
0,ABSTRAK\r\nPenelitian ini bertujuan untuk mene...,"[abstrak, teliti, tuju, teliti, didik, latih, ...",abstrak teliti tuju teliti didik latih pengaru...
2,Abstrak\r\n\r\nTujuan penelitian ini adalah un...,"[abstrak, tuju, teliti, identifikasi, variabel...",abstrak tuju teliti identifikasi variabel atri...
6,"Abstrak\r\n\r\nHening Ary Putra, Pengaruh Ikla...","[abstrak, hening, ary, putra, pengaruh, iklan,...",abstrak hening ary putra pengaruh iklan online...
8,ABSTRAK\r\n\r\n\tPenelitian ini bertujuan untu...,"[abstrak, teliti, tuju, pengaruh, unsur, unsur...",abstrak teliti tuju pengaruh unsur unsur komun...
11,Abstrak\r\n\r\nPendekatan penelitian yang digu...,"[abstrak, dekat, teliti, teliti, dekat, kuanti...",abstrak dekat teliti teliti dekat kuantitatif ...


In [26]:
data.drop(['abstraksi','abstraksi_clean'],axis=1,inplace=True)
data.head()

,abstraksi_akhir
0,abstrak teliti tuju teliti didik latih pengaru...
2,abstrak tuju teliti identifikasi variabel atri...
6,abstrak hening ary putra pengaruh iklan online...
8,abstrak teliti tuju pengaruh unsur unsur komun...
11,abstrak dekat teliti teliti dekat kuantitatif ...


In [27]:
data.to_csv('abstrak_akhir.csv',encoding='utf8', index=False)

# TF - IDF
Term Frequency — Inverse Document Frequency (TF — IDF) adalah suatu metode algoritma yang berguna untuk menghitung bobot setiap kata yang umum digunakan. Metode ini juga terkenal efisien, mudah dan memiliki hasil yang akurat. Metode ini akan menghitung nilai Term Frequency (TF) dan Inverse Document Frequency (IDF) pada setiap token (kata) di setiap dokumen dalam korpus. Secara sederhana, metode TF-IDF digunakan untuk mengetahui berapa sering suatu kata muncul di dalam dokumen. berikut rumus untuk mencari TF

$w_{t d}=\left\{\begin{array}{cl}1+\log _{10} \mathrm{tf}_{\iota d}, & \text { if } \mathrm{tf}_{L d}>0 \\ 0, & \text { if } \mathrm{tf}_{t d}=0\end{array}\right.$

Sedangkan untuk menghitung nilai idf dapat menggunakan rumus sebagai berikut

$\mathrm{idf}_{j}=\log _{10}\left(N / \mathrm{df}_{j}\right)$

In [28]:
data = pd.read_csv("abstrak_akhir.csv", usecols=["abstraksi_akhir"])
data.columns = ["hasil-akhir"]
data

document = data['hasil-akhir']
a=len(document)

# Create a Vectorizer Object
vectorizer = CountVectorizer()

vectorizer.fit(document)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(document)

# Summarizing the Encoded Texts
print("Encoded Document is:")
vector.toarray()

a = vectorizer.get_feature_names()

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tf = tfidf.fit_transform(vectorizer.fit_transform(document)).toarray()

dfb = pd.DataFrame(data=tf,index=list(range(1, len(tf[:,1])+1, )),columns=[a])
dfb


Vocabulary:  {'abstrak': 7, 'teliti': 1593, 'tuju': 1657, 'didik': 350, 'latih': 851, 'pengaruh': 1126, 'parsial': 1101, 'simultan': 1439, 'rangka': 1274, 'tingkat': 1632, 'produktivitas': 1218, 'karyawan': 713, 'meniliti': 958, 'variabel': 1707, 'dominan': 374, 'bukti': 232, 'hipotesis': 563, 'alat': 59, 'analisis': 80, 'regresi': 1293, 'linier': 877, 'ganda': 493, 'bantu': 169, 'spss': 1474, 'hasil': 546, 'uji': 1672, 'nilai': 1033, 'fhitung': 461, 'signifikan': 1429, 'pt': 1237, 'pos': 1184, 'indonesia': 611, 'persero': 1155, 'kantor': 705, 'cabang': 243, 'bangkal': 164, 'koefisien': 763, 'determinasi': 340, 'square': 1475, 'ubah': 1669, 'sebab': 1378, 'sisa': 1446, 'masuk': 938, 'model': 991, 'benar': 195, 'beta': 206, 'identifikasi': 581, 'atribut': 136, 'produk': 1215, 'putus': 1262, 'beli': 193, 'sedap': 1382, 'masako': 934, 'mana': 914, 'metode': 968, 'kuantitatif': 811, 'populasi': 1180, 'ibu': 578, 'camat': 247, 'robatal': 1333, 'sampang': 1352, 'konsumsi': 785, 'teknik': 158

C:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aar,abc,abcs,abd,abidin,abnormal,abnormar,abstrak,abstraksi,abtrak,...,words,wujud,www,xl,yamaha,yoghurt,yudhistira,yustina,zte,zulkifli
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044541,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032029,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025459,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028662,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012640,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028050,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# LSA
LSA adalah teknik matematika/statistika untuk mengekstraksi dan menyimpulkan hubungan kontekstual arti kata yang diaplikasikan pada bagian teks yang dibutuhkan. LSA bisa digunakan untuk menilai esai dengan mengkonversikan esai menjadi matriks-matriks yang diberi nilai pada masing-masing term untuk dicari kesamaan dengan term referensi. LSA pada dasarnya adalah dekomposisi nilai tunggal.

In [29]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(tf)

In [30]:
print(lsa_top)
print(lsa_top.shape)

[[ 0.29230373  0.26273673 -0.0407618  ...  0.08883671  0.17748416
   0.13080332]
 [ 0.55408874 -0.24439974 -0.07574557 ...  0.01807907  0.01838149
   0.06649345]
 [ 0.27635951 -0.12400553 -0.02374709 ... -0.00264652 -0.01142406
   0.21521547]
 ...
 [ 0.2426963  -0.01497963  0.00553116 ... -0.00710184 -0.21731107
  -0.14528226]
 [ 0.27433756  0.46668321 -0.15447698 ...  0.41435018  0.19105162
  -0.13403867]
 [ 0.19237498  0.12244924 -0.01848398 ... -0.07774647 -0.05901344
   0.32698492]]
(127, 10)


In [31]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  29.23037288240089
Topic  1  :  26.273673450243784
Topic  2  :  -4.076180248245132
Topic  3  :  2.828920546844968
Topic  4  :  10.91182198222143
Topic  5  :  0.4486726080136624
Topic  6  :  2.76247719545605
Topic  7  :  8.883671495423446
Topic  8  :  17.74841601015795
Topic  9  :  13.080331819060817


In [32]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1753)
[[ 0.00124324  0.00113136  0.00169704 ...  0.00939831  0.0006322
   0.00159595]
 [ 0.00109735 -0.00019753 -0.0002963  ... -0.0074623  -0.00051088
   0.00257768]
 [ 0.01227807  0.00161162  0.00241743 ... -0.00057695 -0.00026622
   0.01482975]
 ...
 [ 0.0142178   0.00128155  0.00192232 ... -0.00191941  0.00068871
   0.01462865]
 [ 0.00097218  0.00015808  0.00023713 ...  0.00406642  0.00044797
   0.00489578]
 [ 0.00231095  0.00034856  0.00052285 ...  0.00917844 -0.00308777
   0.00355657]]


In [33]:
# most important words for each topic
vocab = vectorizer.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

NameError: name 'vect' is not defined